# Bucle de entrenamiento
Implementaremos el bucle de entrenamiento completo y añadiremos algunas visualizaciones para monitorear el proceso.

1. Función principal de entrenamiento que incluye:
   - Mini-batch gradient descent
   - Monitoreo de métricas (costo y precisión)
   - Evaluación en conjuntos de entrenamiento y prueba
   - Almacenamiento del historial de entrenamiento

2. Funciones auxiliares:
   - `compute_accuracy`: Calcula la precisión de las predicciones
   - `plot_training_history`: Visualiza la evolución del entrenamiento

3. Características importantes:
   - Usa mini-batches para mejor eficiencia y estabilidad
   - Shuffling de datos en cada época
   - Monitoreo regular del progreso
   - Visualizaciones del proceso de entrenamiento

Los hiperparámetros que hemos establecido son:
- Learning rate: 0.01
- Número de épocas: 2000
- Tamaño de batch: 32
- Frecuencia de impresión: cada 100 épocas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.ft_functions import (
    initialize_parameters,
    relu, relu_derivative,
    sigmoid, sigmoid_derivative
)

def train_neural_network(X_train, Y_train, X_test, Y_test, layer_dims, 
                        learning_rate=0.01, num_epochs=2000, batch_size=32, 
                        print_every=100):
    """
    Entrena la red neuronal
    
    Args:
        X_train, Y_train: Datos de entrenamiento
        X_test, Y_test: Datos de prueba
        layer_dims: Lista con dimensiones de cada capa
        learning_rate: Tasa de aprendizaje
        num_epochs: Número de épocas
        batch_size: Tamaño del mini-batch
        print_every: Cada cuántas épocas imprimir resultados
    
    Returns:
        dict: Historia del entrenamiento
        dict: Parámetros finales
    """
    np.random.seed(42)
    parameters = initialize_parameters(layer_dims)
    m = X_train.shape[1]  # número de ejemplos
    
    # Para almacenar el historial
    history = {
        'train_cost': [],
        'test_cost': [],
        'train_accuracy': [],
        'test_accuracy': []
    }
    
    # Número de mini-batches
    n_batches = max(m // batch_size, 1)
    
    for epoch in range(num_epochs):
        epoch_cost = 0
        
        # Crear permutación aleatoria de los índices
        permutation = np.random.permutation(m)
        X_shuffled = X_train[:, permutation]
        Y_shuffled = Y_train[:, permutation]
        
        # Mini-batch training
        for batch in range(n_batches):
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, m)
            
            X_batch = X_shuffled[:, start_idx:end_idx]
            Y_batch = Y_shuffled[:, start_idx:end_idx]
            
            # Forward propagation
            cache, A3 = forward_propagation(X_batch, parameters)
            
            # Calcular costo
            batch_cost = compute_cost(A3, Y_batch)
            epoch_cost += batch_cost
            
            # Backward propagation
            gradients = backward_propagation(X_batch, Y_batch, parameters, cache)
            
            # Actualizar parámetros
            parameters = update_parameters(parameters, gradients, learning_rate)
        
        epoch_cost /= n_batches
        
        if epoch % print_every == 0:
            # Evaluar en conjunto de entrenamiento
            train_cache, train_predictions = forward_propagation(X_train, parameters)
            train_cost = compute_cost(train_predictions, Y_train)
            train_accuracy = compute_accuracy(train_predictions, Y_train)
            
            # Evaluar en conjunto de prueba
            test_cache, test_predictions = forward_propagation(X_test, parameters)
            test_cost = compute_cost(test_predictions, Y_test)
            test_accuracy = compute_accuracy(test_predictions, Y_test)
            
            # Guardar métricas
            history['train_cost'].append(train_cost)
            history['test_cost'].append(test_cost)
            history['train_accuracy'].append(train_accuracy)
            history['test_accuracy'].append(test_accuracy)
            
            print(f"Época {epoch}/{num_epochs}")
            print(f"  Costo entrenamiento: {train_cost:.4f}")
            print(f"  Precisión entrenamiento: {train_accuracy:.4f}")
            print(f"  Costo prueba: {test_cost:.4f}")
            print(f"  Precisión prueba: {test_accuracy:.4f}")
    
    return history, parameters

def compute_accuracy(predictions, Y):
    """
    Calcula la precisión de las predicciones
    """
    predictions = predictions > 0.5
    return np.mean(predictions == Y)

def plot_training_history(history):
    """
    Visualiza el historial de entrenamiento
    """
    epochs = range(0, len(history['train_cost']))
    
    plt.figure(figsize=(15, 5))
    
    # Gráfico de costo
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_cost'], label='Entrenamiento')
    plt.plot(epochs, history['test_cost'], label='Prueba')
    plt.title('Evolución del Costo')
    plt.xlabel('Épocas')
    plt.ylabel('Costo')
    plt.legend()
    plt.grid(True)
    
    # Gráfico de precisión
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_accuracy'], label='Entrenamiento')
    plt.plot(epochs, history['test_accuracy'], label='Prueba')
    plt.title('Evolución de la Precisión')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Cargar datos del notebook anterior
# Asumimos que X_train_scaled, X_test_scaled, y_train, y_test están disponibles

# Preparar los datos en el formato correcto (asegurarse de que sean matrices 2D)
X_train = X_train_scaled.T  # Transponer para tener formato (n_features, n_samples)
X_test = X_test_scaled.T
Y_train = y_train.reshape(1, -1)  # Reshape para tener formato (1, n_samples)
Y_test = y_test.reshape(1, -1)

# Definir arquitectura de la red
layer_dims = [30, 16, 8, 1]

# Entrenar la red
history, trained_parameters = train_neural_network(
    X_train, Y_train, X_test, Y_test,
    layer_dims,
    learning_rate=0.01,
    num_epochs=2000,
    batch_size=32,
    print_every=100
)

# Visualizar el progreso del entrenamiento
plot_training_history(history)

# Evaluar modelo final
_, final_train_predictions = forward_propagation(X_train, trained_parameters)
_, final_test_predictions = forward_propagation(X_test, trained_parameters)

print("\nResultados finales:")
print(f"Precisión en entrenamiento: {compute_accuracy(final_train_predictions, Y_train):.4f}")
print(f"Precisión en prueba: {compute_accuracy(final_test_predictions, Y_test):.4f}")